In [32]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import time
from lightgbm import LGBMClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
#the ones under here are for heat-map
import seaborn as sns
import matplotlib.pyplot as plt


Importing the dataset

In [33]:
dataset = pd.read_csv("weatherAUS.csv")
dataset.describe()


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [34]:
dataset.isnull().sum()


Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [35]:
dataset=dataset.drop(['Date'],axis=1)
dataset=dataset.dropna(how='any')
dataset.isnull().sum()


Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [36]:
categorical, numerical = [], []

for i in dataset.columns:

    if dataset[i].dtype == 'object':
        categorical.append(i)
    else:
        numerical.append(i)

#separate categorical and numerical columns
categorical=dataset[['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']]
numerical=dataset[['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']]
label_encoder = LabelEncoder()
for i in categorical:
    dataset[i] = label_encoder.fit_transform(dataset[i])

dataset.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 56420 entries, 6049 to 142302
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       56420 non-null  int32  
 1   MinTemp        56420 non-null  float64
 2   MaxTemp        56420 non-null  float64
 3   Rainfall       56420 non-null  float64
 4   Evaporation    56420 non-null  float64
 5   Sunshine       56420 non-null  float64
 6   WindGustDir    56420 non-null  int32  
 7   WindGustSpeed  56420 non-null  float64
 8   WindDir9am     56420 non-null  int32  
 9   WindDir3pm     56420 non-null  int32  
 10  WindSpeed9am   56420 non-null  float64
 11  WindSpeed3pm   56420 non-null  float64
 12  Humidity9am    56420 non-null  float64
 13  Humidity3pm    56420 non-null  float64
 14  Pressure9am    56420 non-null  float64
 15  Pressure3pm    56420 non-null  float64
 16  Cloud9am       56420 non-null  float64
 17  Cloud3pm       56420 non-null  float64
 18  Te

In [37]:
y=dataset['RainTomorrow']
X=dataset.drop(['RainTomorrow'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()

scaler.fit(X_train)
X_train_s = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)

scaler.fit(X_test)
X_test_s = pd.DataFrame(scaler.fit_transform(X_test),columns = X_test.columns)


def max_value(df, variable, top):
    return np.where(df[variable]>top, top, df[variable])

for df in [X_train_s, X_test_s]:
    df['Rainfall'] = max_value(df, 'Rainfall', 3.2)
    df['Evaporation'] = max_value(df, 'Evaporation', 2.8)
    df['WindSpeed9am'] = max_value(df, 'WindSpeed9am', 5)
    df['WindSpeed3pm'] = max_value(df, 'WindSpeed3pm', 5)
    df['WindGustSpeed'] =max_value(df,'WindGustSpeed', 4)


In [38]:
#Gaıssian Naive Bayes
gnb = GaussianNB()
start_time1=time.time()
pred = gnb.fit(X_train_s, y_train).predict(X_test_s)

end_time1=time.time()
predictions1= gnb.predict(X_test_s)
classification_report(y_test,predictions1)


total = (X_test_s.shape[0])
mislabelled = (y_test != predictions1).sum()

print("Accuracy percentage for Naive Bayes is :")
print((total-mislabelled)/total)
print(end_time1-start_time1)


Accuracy percentage for Naive Bayes is :
0.7953739808578518
0.0388948917388916


In [39]:
classification_report(y_test, predictions1)


'              precision    recall  f1-score   support\n\n           0       0.91      0.82      0.86      8854\n           1       0.52      0.70      0.60      2430\n\n    accuracy                           0.80     11284\n   macro avg       0.71      0.76      0.73     11284\nweighted avg       0.82      0.80      0.81     11284\n'

In [40]:

scores_gnb=cross_val_score(gnb, X, y, cv=10)

("accuracy with a standard deviation of" , (scores_gnb.mean(), scores_gnb.std()))


('accuracy with a standard deviation of',
 (0.7961006735200283, 0.04600928041183562))

In [24]:

model = RandomForestClassifier()
time_start= time.time()
model.fit(X_train_s, y_train)
time_end= time.time()

predictions = model.predict(X_test_s)
classification_report(y_test, predictions)


'              precision    recall  f1-score   support\n\n           0       0.88      0.95      0.92      8854\n           1       0.76      0.55      0.64      2430\n\n    accuracy                           0.87     11284\n   macro avg       0.82      0.75      0.78     11284\nweighted avg       0.86      0.87      0.86     11284\n'

In [25]:
 
accuracy_score(y_test, predictions)
accuracy_score(y_test, predictions)

"learning time for holdout method", (time_end-time_start)



('learning time for holdout method', 5.599241733551025)

In [26]:
time_start1=time.time()
param_grid = {
  'n_estimators': [100, 200],
    'max_features': ['auto'],
    'max_depth' : [4,5,8,10],
   'criterion' :['gini', 'entropy']
}
RFC = RandomForestClassifier()
cv_RFC = GridSearchCV(estimator=RFC, param_grid=param_grid, cv=10)
best_model= cv_RFC.fit(X_train_s, y_train)
time_end1=time.time()
bestparameters=cv_RFC.best_params_
bestparameters


{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 200}

In [27]:

"learning time for gridsearch", (time_end1-time_start1)



('learning time for gridsearch', 606.4203310012817)

In [28]:
clf=RandomForestClassifier(criterion='gini',
                                                                              max_depth=10,
                                                                              max_features='auto',
                                                                              n_estimators=200)

time_start2=time.time()
clf.fit(X_train_s, y_train)
time_end2=time.time()
"learning time with best parameters",time_end2-time_start2 


('learning time with best parameters', 7.202383279800415)

In [31]:
scores =cross_val_score(clf, X, y, cv=10)
"accuracy with a standard deviation of " , (scores.mean(), scores.std())



('accuracy with a standard deviation of ',
 (0.8409606522509749, 0.023103976799738773))

In [ ]:
y_pred = clf.predict(X_test_s)
y_pred = clf.predict(X_test_s)
confusion_matrix(y_test, y_pred)


In [ ]:

classification_report(y_test,y_pred)
classification_report(y_test,y_pred)


In [ ]:
accuracy_score(y_test, y_pred)


In [ ]:

"f1 score is :",f1_score(y_test, y_pred)


In [ ]:

(y_test, y_pred)
precision_score(y_test, y_pred)


In [ ]:

recall_score(y_test, y_pred)



In [ ]:
logreg=LogisticRegression()
starttime=time.time()
logreg.fit(X_train_s,y_train)
endtime=time.time()
predictions2=logreg.predict(X_test_s)
classification_report(y_test,predictions2)



In [ ]:
accuracy_score(y_test,predictions2)



In [ ]:
 logistic regression
"learning time logistic regression for the holdout method" (endtime-starttime)

In [ ]:
svclassifier = SVC(kernel='rbf')
start_time_svm=time.time()
svclassifier.fit(X_train_s, y_train)
end_time_svm=time.time()
predictions_svm=svclassifier.predict(X_test_s)
print("holdout method - classification report for SVM ")

print( classification_report(y_test, predictions_svm))

print("accuracy score for holdout method",accuracy_score(y_test, predictions_svm))

print("learning time for holdout method", (end_time_svm-start_time_svm))



In [41]:
lightgbmc = LGBMClassifier(n_estimators=500,max_depth=6)
timeyy=time.time()
lightgbmc.fit(X_train_s, y_train)
timeyy2=time.time()
(timeyy2-timeyy)


0.7179038524627686

In [42]:

y_pred2 = lightgbmc.predict(X_test_s)
classification_report(y_test, y_pred2)



'              precision    recall  f1-score   support\n\n           0       0.90      0.94      0.92      8854\n           1       0.75      0.60      0.67      2430\n\n    accuracy                           0.87     11284\n   macro avg       0.82      0.77      0.79     11284\nweighted avg       0.86      0.87      0.86     11284\n'

In [43]:
"Accuracy of the LightGBM Model is:", accuracy_score(y_test,y_pred2)



('Accuracy of the LightGBM Model is:', 0.8702587734845799)

In [44]:
confusion_matrix(y_test, y_pred2)



array([[8358,  496],
       [ 968, 1462]], dtype=int64)

In [45]:
lgb=LGBMClassifier()

parameters = {'num_leaves':[20,40,60,80,100], 'min_child_samples':[5,10,15],'max_depth':[-1,5,10,20],
             'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}

clf3=GridSearchCV(lgb,parameters,cv=5)

clf3.fit(X=X_train_s, y=y_train)
print(clf3.best_params_)


{'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 15, 'num_leaves': 60, 'reg_alpha': 0.01}


In [50]:

clf3best=LGBMClassifier(num_leaves= 60, min_child_samples=15,max_depth=10,
             learning_rate=0.1,reg_alpha=0.03)

clf3best.fit(X_train_s, y_train)
scores3 =cross_val_score(clf3best, X, y, cv=5)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores3.mean(), scores3.std()))



0.81 accuracy with a standard deviation of 0.06


In [51]:
y_pred3 = clf3best.predict(X_test_s)
classification_report(y_test,y_pred3)
          

'              precision    recall  f1-score   support\n\n           0       0.89      0.94      0.92      8854\n           1       0.74      0.59      0.66      2430\n\n    accuracy                           0.87     11284\n   macro avg       0.82      0.77      0.79     11284\nweighted avg       0.86      0.87      0.86     11284\n'

In [52]:

accuracy_score(y_test,y_pred3)

0.8677773839064161